In [1]:
#importing dataset
import pandas as pd
df= pd.read_csv(r"C:\Users\shaha\Desktop\IBS\train.csv")
df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [2]:
#preprocessing 
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
import nltk
from nltk.corpus import stopwords
stopwords = nltk.corpus.stopwords.words('english')
from nltk.tokenize import word_tokenize
import string

df['new_tweet'] = df.tweet.str.replace('@user', '')
df['new_tweet'] = df['new_tweet'].str.replace("[^a-zA-Z#]", " " )
df['new_tweet'] = df['new_tweet'].str.replace("#", " ")
df['new_tweet'] = df['new_tweet'].apply(lambda x: x.split())
df['new_tweet']= df['new_tweet'].apply(lambda x: [stemmer.stem(i) for i in x])
def process(text):
    # Check characters to see if they are in punctuation
    nopunc = set(char for char in list(text) if char not in string.punctuation)
    # Join the characters to form the string.
    nopunc = " ".join(nopunc)
    # remove any stopwords if present
    return [word for word in nopunc.lower().split() if word.lower() not in stopwords]
df['new_tweet'] = df['new_tweet'].apply(process)
df=df.drop(columns='tweet')
df=df.drop(columns='id')
df.head()


<ipython-input-2-d574088bc6ff>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df['new_tweet'] = df['new_tweet'].str.replace("[^a-zA-Z#]", " " )


,label,new_tweet
0,0,"[run, dysfunct, father, kid, drag, selfish]"
1,0,"[credit, van, disapoint, pdx, use, wheelchair,..."
2,0,"[bihday, majesti]"
3,0,"[u, time, love, take, model, ur]"
4,0,"[factsguid, societi, motiv]"


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
x_train, x_test, y_train, y_test =train_test_split(df["new_tweet"], df["label"], test_size = 0.2, random_state = 42)


In [4]:
import numpy as np
split=np.array_split(x_train, 10)
label=np.array_split(y_train,10)

In [5]:
#Queriying
from scipy.stats import entropy
import numpy as np
def multi_argmax(values, n_instances= 100):
    max_idx = np.argpartition(-values, n_instances-1, axis=0)[:n_instances]
    return max_idx
def entropy_sampling( X,n_instances= 100):
    ent=np.transpose(entropy(np.transpose(X)))
    return multi_argmax(ent, n_instances=n_instances)


In [6]:
import scipy.sparse as sp
import pandas as pd
def add_data(x,y):
    x=sp.vstack((x,y))
    return x
def add_label(x,y):
    x=np.concatenate((x,y))
    return x
def fit_model(x,y):
    model = RandomForestClassifier()
    model.fit(x,y)
    return model

In [7]:
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
x_train_counts = count_vect.fit_transform(split[0].apply(lambda x: ' '.join(x)))
x_train_tfidf = transformer.fit_transform(x_train_counts)
label[0]=label[0].to_numpy()
model=fit_model(x_train_tfidf,label[0])
x_test_counts = count_vect.transform(x_test.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
predictions = model.predict(x_test_tfidf)
print(accuracy_score(y_test,predictions)*100)


94.10292507430002


In [8]:
for i in range(1,10):
    x_counts = count_vect.transform(split[i].apply(lambda x: ' '.join(x)))
    x_tfidf = transformer.transform(x_counts)
    sample=model.predict_proba(x_tfidf)
    tweet_id=entropy_sampling(sample)
    #label_tweet=retrieve_rows(x_tfidf,tweet_id)
    y=[]
    #Teaching 
    for k in tweet_id:
      j=label[i].iloc[k]
      lb=j.item()
      y.append(lb)
    x_train_tfidf=add_data(x_train_tfidf,x_tfidf[tweet_id])
    label[0]=add_label(label[0],y)
    model=fit_model(x_train_tfidf,label[0])
    
x_test_counts = count_vect.transform(x_test.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
predictions = model.predict(x_test_tfidf)
print(accuracy_score(y_test,predictions)*100)

95.38557797591115


In [9]:
#Not using Active learning

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
x_train, x_test, y_train, y_test =train_test_split(df["new_tweet"], df["label"], test_size = 0.2, random_state = 42)
print("Training data counts:",len(x_train))
print("Testing data counts:",len(x_test))
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
x_train_counts = count_vect.fit_transform(x_train.apply(lambda x: ' '.join(x)))
x_train_tfidf = transformer.fit_transform(x_train_counts)
x_test_counts = count_vect.transform(x_test.apply(lambda x: ' '.join(x)))
x_test_tfidf = transformer.transform(x_test_counts)
model = RandomForestClassifier(n_estimators=200)
model.fit(x_train_tfidf,y_train)
predictions = model.predict(x_test_tfidf)
print(accuracy_score(y_test,predictions)*100)

Training data counts: 25569
Testing data counts: 6393
96.13639918661036
